# Optimal Sensor Placement for Modal Analysis

**LADPackage Demo: Optimal Sensor Placement**

This notebook demonstrates optimal sensor placement methods for structural health monitoring using modal analysis. The example compares two optimization approaches:

1. **Fisher Information Method**: Maximizes determinant of Fisher Information Matrix using Effective Independence algorithm
2. **Maximum Norm Method**: Maximizes weighted norm of modal responses with distance constraints

Both methods determine optimal locations for 12 sensors on a structural system to maximize modal observability.

---

**Reference**: Auto-Generated Script by mFUSE (3/19/2016)  
**Authors**: Eric Flynn, Dustin Harvey

**Educational Goals**:
- Understand sensor placement optimization criteria
- Compare Fisher Information vs Maximum Norm approaches
- Visualize optimal sensor configurations on structural geometry
- Learn modal observability concepts for SHM system design

## Setup and Imports

Import necessary modules and set up the path for SHMTools and LADPackage utilities access.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import sys
from pathlib import Path

# Add project root to path for imports
notebook_dir = Path().resolve()
project_root = notebook_dir.parent.parent
sys.path.insert(0, str(project_root))

print(f"Notebook directory: {notebook_dir}")
print(f"Project root: {project_root}")

# Import SHMTools modules
from examples.data.data_imports import import_modal_osp_shm
from shmtools.modal.osp import osp_fisher_info_eiv_shm, osp_max_norm_shm

# Import LADPackage plotting utilities
from LADPackage.utils import plot_nodal_response, plot_sensors_with_mesh

print("\nSHMTools and LADPackage imports successful!")
print("Ready for optimal sensor placement analysis.")

## Step 1: Import Modal Optimal Sensor Placement Dataset

Import Modal Optimal Sensor Placement Data including finite element model geometry, mode shapes, and degree-of-freedom definitions.

**Dataset Contents**:
- **Node layout**: 3D coordinates of finite element nodes
- **Elements**: Connectivity matrix defining mesh topology 
- **Mode shapes**: Modal response vectors for structural dynamics
- **Response DOFs**: Degree-of-freedom definitions linking modal data to geometry

In [ ]:
# Import Modal Optimal Sensor Placement Data
[node_layout, elements, mode_shapes, resp_dof] = import_modal_osp_shm()

print(f"Modal OSP data loaded successfully:")
print(f"  Node layout shape: {node_layout.shape}")
print(f"  Elements shape: {elements.shape}")
print(f"  Mode shapes shape: {mode_shapes.shape}")
print(f"  Response DOF shape: {resp_dof.shape}")

print(f"\nStructural Model Summary:")
print(f"  Number of nodes: {node_layout.shape[1]}")
print(f"  Number of elements: {elements.shape[1]}")
print(f"  Number of DOFs: {mode_shapes.shape[0]}")
print(f"  Number of modes: {mode_shapes.shape[1]}")

# Display geometry bounds
x_range = [node_layout[0,:].min(), node_layout[0,:].max()]
y_range = [node_layout[1,:].min(), node_layout[1,:].max()]  
z_range = [node_layout[2,:].min(), node_layout[2,:].max()]
print(f"\nGeometry bounds:")
print(f"  X: {x_range[0]:.1f} to {x_range[1]:.1f}")
print(f"  Y: {y_range[0]:.1f} to {y_range[1]:.1f}")
print(f"  Z: {z_range[0]:.1f} to {z_range[1]:.1f}")

## Step 2: Plot Nodal Response (Mode 3)

Visualize the nodal response for mode 3 to understand the structural dynamics and modal characteristics.

**Nodal Response Visualization**:
- **3D mesh**: Shows finite element model geometry
- **Color mapping**: Response amplitude at each node
- **Modal pattern**: Reveals characteristic deformation shape

In [ ]:
# Plot nodal response for mode 3
mode_number_1 = 3

axes_handle_1 = plot_nodal_response(node_layout, mode_shapes, resp_dof, elements, mode_number_1)

plt.title(f'Mode {mode_number_1} Nodal Response\nStructural Modal Analysis')
plt.tight_layout()
plt.show()

print(f"\nMode {mode_number_1} visualization completed")
print(f"Color map shows response amplitude distribution across structure")
print(f"This mode shape will influence optimal sensor placement strategy")

## Step 3: Plot Nodal Response (Mode 10) 

Visualize the nodal response for mode 10 to compare with mode 3 and understand higher-order modal characteristics.

**Higher-Order Modes**:
- **Increased complexity**: More variation across structure
- **Local effects**: Higher spatial frequency content
- **Sensor requirements**: May need different optimal locations

In [ ]:
# Plot nodal response for mode 10
mode_number_2 = 10

axes_handle_2 = plot_nodal_response(node_layout, mode_shapes, resp_dof, elements, mode_number_2)

plt.title(f'Mode {mode_number_2} Nodal Response\nHigher-Order Modal Pattern')
plt.tight_layout()
plt.show()

print(f"\nMode {mode_number_2} visualization completed")
print(f"Higher-order modes show increased spatial complexity")
print(f"Comparison with Mode {mode_number_1} reveals different response patterns")

## Step 4: OSP Fisher Information EIV

Calculate the 12 optimal DOFs to place sensors by maximizing the determinant of the Fisher Information Matrix using the Equivalent Independence method.

**Fisher Information Method**:
- **Objective**: Maximize det(Fisher Information Matrix)
- **Algorithm**: Effective Independence (EI) - greedy sequential selection
- **Criterion**: Maximizes information content for modal parameter estimation
- **Advantage**: Theoretically optimal for parameter estimation accuracy

In [ ]:
# OSP using Fisher Information Matrix with Effective Independence method
num_sensors_fisher = 12

[optimal_list_fisher, det_q] = osp_fisher_info_eiv_shm(num_sensors_fisher, mode_shapes, None)

print(f"Fisher Information EIV optimization completed:")
print(f"  Number of optimal sensors: {num_sensors_fisher}")
print(f"  Determinant of Q matrix: {det_q}")
print(f"  Optimal DOF indices: {optimal_list_fisher}")

# Display sensor statistics
print(f"\nOptimal sensor DOFs (Fisher Information):")
for i, dof in enumerate(optimal_list_fisher):
    if dof-1 < len(resp_dof):  # Convert to 0-based indexing
        node_id, direction = resp_dof[dof-1, :]
        print(f"  Sensor {i+1:2d}: DOF {dof:3d} -> Node {node_id:.0f}, Direction {direction:.0f}")
    else:
        print(f"  Sensor {i+1:2d}: DOF {dof:3d} (index out of range)")

print(f"\nFisher Information criterion maximizes modal parameter estimation accuracy")

## Step 5: Plot Sensors With Mesh (Fisher Information)

Visualize the optimal sensor locations determined by the Fisher Information method overlaid on the structural mesh.

**Visualization Features**:
- **3D wireframe mesh**: Structural geometry 
- **Sensor locations**: Red markers at optimal positions
- **Sensor numbering**: Labels for identification
- **Spatial distribution**: Shows coverage across structure

In [ ]:
# Plot optimal sensors on mesh (Fisher Information method)
[axes_handle_fisher_1, axes_handle_fisher_2, sensor_handle_fisher] = plot_sensors_with_mesh(
    elements, node_layout, optimal_list_fisher, resp_dof)

plt.title(f'Optimal Sensor Placement: Fisher Information Method\n{num_sensors_fisher} Sensors, det(Q) = {det_q:.2e}')
plt.tight_layout()
plt.show()

print(f"\nFisher Information sensor placement visualization completed")
print(f"Red markers show optimal locations for {num_sensors_fisher} sensors")
print(f"Sensor distribution maximizes information for modal parameter estimation")

## Step 6: OSP Using Maximum Norm

Calculate the 12 optimal DOFs using the Maximum Norm method with linear mode weighting and minimum separation constraints.

**Maximum Norm Method**:
- **Objective**: Maximize weighted norm of modal responses  
- **Weighting**: Linear weights (13, 12, 11, ..., 1) favor lower modes
- **Distance constraint**: Minimum separation of 20 units between sensors
- **"Dueling"**: Eliminates sensors that are too close together
- **Advantage**: Ensures good spatial distribution and practical placement

In [ ]:
# OSP using Maximum Norm method with constraints
num_sensors_norm = 12
weights = np.array(range(13, 0, -1))  # Linear weights: 13, 12, 11, ..., 1
dueling_distance = 20  # Minimum separation distance

[optimal_list_norm] = osp_max_norm_shm(
    num_sensors_norm, mode_shapes, weights, dueling_distance, resp_dof, node_layout)

print(f"Maximum Norm optimization completed:")
print(f"  Number of optimal sensors: {num_sensors_norm}")
print(f"  Mode weights: {weights}")
print(f"  Minimum separation distance: {dueling_distance}")
print(f"  Optimal DOF indices: {optimal_list_norm}")

# Display sensor statistics
print(f"\nOptimal sensor DOFs (Maximum Norm):")
for i, dof in enumerate(optimal_list_norm):
    if dof-1 < len(resp_dof):  # Convert to 0-based indexing
        node_id, direction = resp_dof[dof-1, :]
        print(f"  Sensor {i+1:2d}: DOF {dof:3d} -> Node {node_id:.0f}, Direction {direction:.0f}")
    else:
        print(f"  Sensor {i+1:2d}: DOF {dof:3d} (index out of range)")

print(f"\nMaximum Norm method ensures good spatial distribution with distance constraints")

## Step 7: Plot Sensors With Mesh (Maximum Norm)

Visualize the optimal sensor locations determined by the Maximum Norm method overlaid on the structural mesh.

**Comparison with Fisher Information**:
- **Different distribution**: May show different spatial patterns
- **Distance constraints**: Enforced minimum separation
- **Mode weighting effects**: Lower modes have higher influence
- **Practical considerations**: Better for real-world sensor deployment

In [ ]:
# Plot optimal sensors on mesh (Maximum Norm method)
[axes_handle_norm_1, axes_handle_norm_2, sensor_handle_norm] = plot_sensors_with_mesh(
    elements, node_layout, optimal_list_norm, resp_dof)

plt.title(f'Optimal Sensor Placement: Maximum Norm Method\n{num_sensors_norm} Sensors, Min Distance = {dueling_distance}')
plt.tight_layout()
plt.show()

print(f"\nMaximum Norm sensor placement visualization completed")
print(f"Red markers show optimal locations for {num_sensors_norm} sensors")
print(f"Sensor distribution balances modal observability with practical constraints")

## Analysis Summary

This LADPackage optimal sensor placement demo demonstrates two fundamental approaches for designing sensor networks in structural health monitoring:

### Key Results:

1. **Modal Visualization**: Successfully visualized mode shapes 3 and 10 showing different spatial complexity
2. **Fisher Information Method**: Determined 12 optimal sensors maximizing modal parameter estimation accuracy
3. **Maximum Norm Method**: Computed 12 optimal sensors with spatial distribution constraints and mode weighting

### Method Comparison:

| Method | Objective | Advantages | Considerations |
|--------|-----------|------------|----------------|
| **Fisher Information** | Maximize det(FIM) | Theoretically optimal for estimation | May cluster sensors |
| **Maximum Norm** | Maximize weighted response | Good spatial distribution | Requires distance tuning |

### Practical Implications:

- **Fisher Information**: Best for parameter estimation accuracy when sensor clustering is acceptable
- **Maximum Norm**: Better for practical deployments requiring distributed coverage
- **Hybrid approaches**: Can combine both criteria for balanced solutions

### SHM System Design Guidelines:

1. **Modal coverage**: Ensure sensors can observe critical modes
2. **Spatial distribution**: Balance coverage with installation constraints
3. **Mode weighting**: Prioritize modes based on damage sensitivity
4. **Physical constraints**: Consider accessibility and wiring requirements

---

**Note**: This analysis provides the foundation for designing effective SHM sensor networks using modal-based optimization criteria.